In [4]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import uncertainties as unc
from scipy import stats
from scipy.stats import norm 
from scipy.stats import uniform
from scipy.stats import binom
from scipy.stats import poisson
from scipy.stats import cauchy
import scipy.constants as const
from tabulate import tabulate
import math
from matplotlib import cm
import matplotlib.cm as cm
import seaborn as sns
from scipy.optimize import minimize
from iminuit import Minuit
import pickle
import numpy as np
from pyuvdata import UVData
%matplotlib inline

In [5]:
path_to_data_file = './Hera/'
uv = UVData()
uv.read('./zen.2459122.34011.mini.sum (2).uvh5')
data = uv.data_array
freqs = uv.freq_array[0]*1e-6
print('This data array has shape:')
print('(# baselines*times, # spectral windows, # frequencies, # polarizations)')
print(np.shape(data))
print(data.ndim)


This data array has shape:
(# baselines*times, # spectral windows, # frequencies, # polarizations)
(13320, 1, 1536, 1)
4


### Define function to find amplitude differences between adjacent time slices for all pairs

In [7]:
# # Amplitude diff between adjacent slices
# def amplitude_diff(uvdata):
#     # Get unique baselines with visibilities
#     baseline_pairs = uvdata.get_baseline_nums()
#     # Initialize place to store amplitude diffs
#     amp_diffs = {} 
#     # Iterate over pairs to get amplitude diffs
#     for baseline in baseline_pairs:
#         data_apair = uvdata.get_data(baseline)
#         amplitudes = np.abs(data_apair)
#         difference = np.abs(np.diff(amplitudes, axis=0))
#         amp_diffs[baseline] = difference
#     return amp_diffs
        
        

### DEfine threshold function that includes different methods with default as percentile

### Define Function to filter through each pairing and plot freq. vs. avg amp before and after

In [10]:
# Amplitude diff between adjacent slices
def amplitude_diff(uvdata):
    # Get unique baselines with visibilities
    baseline_pairs = uvdata.get_baseline_nums()
    # Initialize place to store amplitude diffs
    amp_diffs = {} 
    # Iterate over pairs to get amplitude diffs
    for baseline in baseline_pairs:
        data_apair = uvdata.get_data(baseline)
        amplitudes = np.abs(data_apair)
        difference = np.abs(np.diff(amplitudes, axis=0))
        amp_diffs[baseline] = difference
    return amp_diffs

def filter(data):
    baseline_pairs = data.get_baseline_nums()
    # Call amplitude difference function to get aarray with amplitude differences
    amplitude_differences = amplitude_diff(data)
    # Initialize storing of filtered data for each pair
    filtered_data = {}
    # Threshold function
    def find_threshold(d, method='quantile', q=0.05):
            if method == 'quantile':
                threshold = np.quantile(d, axis=0, q=q)
            else:
                raise ValueError("Unsupported threshold method.")
            return threshold
    # Iterate through each baseline pair to determine threshold then filter data
    for baseline in baseline_pairs:
        # Get antenna numbers
        antenna1, antenna2 = data.baseline_to_antnums(baseline)
        
        #### Determine threshold
        # Threshold function
        # def find_threshold(d, method='quantile', q=0.05):
        #         if method == 'quantile':
        #             threshold = np.quantile(d, axis=0, q=q)
        #         else:
        #             raise ValueError("Unsupported threshold method.")
        #         return threshold
        
        
       
        ### Filter
        data_pair = data.get_data(baseline)
        avg_diff_before = np.mean(amplitude_differences[baseline], axis=0)
        threshold = find_threshold(avg_diff_before, method='quantile', q=0.99)
        freqs_reject = np.any(amplitude_differences[baseline] >= threshold, axis=0) # List of frequencies to remove based on threshold
        rejected_indices = np.where(freqs_reject)[0] # Location of frequencies to remove 
        amplitudes_filtered = np.delete(np.abs(data_pair), rejected_indices, axis=1) # Delete data corresponding to indices to reject
        freqs = data.freq_array[0]*1e-6 # Original frequencies
        freqs_filtered = np.delete(freqs, rejected_indices) # Delete frequencies correspondinf to indices to reject
        # print(amplitude_differences[baseline])
        # Store filtered data for each pair
        filtered_data[baseline] = amplitudes_filtered
        
        avg_diff_after = np.mean(np.abs(np.diff(np.abs(amplitudes_filtered), axis=0)), axis=0)
        
        
        print("freqs_filtered shape:", freqs_filtered.shape)
        print("amplitudes_filtered shape:", amplitudes_filtered.shape)
        if len(freqs_filtered) == 0 or amplitudes_filtered.size == 0:
            print(f"No data left after filtering for baseline ({antenna1}, {antenna2}).")
            continue
        print("Threshold for baseline:", threshold)
      
        
        # plotting diff
        plt.figure(figsize=(10, 4))
        plt.plot(freqs, avg_diff_before, alpha=0.4, color='k', label=f"Baseline Pair ({antenna1}, {antenna2}) Before Filtering")
        plt.plot(freqs_filtered, avg_diff_after, alpha=0.2, color='r', label=f"Baseline Pair ({antenna1}, {antenna2}) After Filtering")
        plt.xlabel("Frequency")
        plt.ylabel("Average Amplitude")
        plt.legend()
        plt.title(f"Frequency vs Avg. Amplitude for Baseline Pair ({antenna1}, {antenna2})")
        plt.show()
    return filtered_data
    
    
    

In [11]:
# n = filter(uv)